<a href="https://colab.research.google.com/github/WeiHao-19/weihaoi.github.io/blob/gh-pages/%E7%AE%80%E5%8D%95%E7%9A%84convnet%E5%92%8Cdenses%E6%A8%A1%E5%9E%8B%E5%AF%B9%E6%AF%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import keras
keras.__version__

'2.4.3'

In [3]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


- 下一步是将最后的输出张量**conv2d_2 (Conv2D)** ［大小为 (3, 3, 64)］输入到一个密集连接分类器网络中，即 **Dense 层**的堆叠。这些分类器可以处理 1D 向量，而当前的输出是 3D 张量。
- 首先，我们需要(**压扁**)将 3D 输出展平为 1D，然后在上面添加几个 Dense 层。

In [5]:
# 在卷积神经网络上添加分类器
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [6]:
# 为了进行 10 类别分类，所以最后一层使用带 10 个输出的 softmax 激活。
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 19s 3ms/step - loss: 0.3833 - accuracy: 0.8780
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0486 - accuracy: 0.9846
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0309 - accuracy: 0.9905
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0246 - accuracy: 0.9927
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0203 - accuracy: 0.9943


In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)


313/313 [==============================] - 1s 2ms/step - loss: 0.0243 - accuracy: 0.9932


In [10]:
test_acc

0.9932000041007996

我们在这里最终的训练模型在（前所未见的）测试集上（densely-connected network）密集连接网络的测试精度为 97.8%，但这个简单卷积神经网络的测试精度达到了99.3%，我们将错误率降低了 68%（相对比例）。相当不错！
- 有个问题：**与密集连接模型相比，为什么这个简单卷积神经网络的效果这么好？**
- （需要深入了解 Conv2D 层和 MaxPooling2D 层的作用。）